# Multinomial and Bernoulli Naive Bayes
For understanding Multinomial and Bernoulli Naive Bayes, we will start with a small example and understand the end to end process. In another notebook, we will build a full-fledged email spam classifier.

To start with, let's take a few sentences and classify them in two different classes - education or cinema. Each sentence will represent one document. In real-world cases, a document be any piece of text such as an email, a news article, a book review, a tweet etc. The analysis and the algorithm involved doesn’t depend on the type of document we use.

The notebook is divided into the following sections:

- Importing and preprocessing data
- Building the model: Multinomial Naive Bayes
- Building the model: Bernoulli Naive Bayes

## 1. Importing and Preprocessing Data
Let us first look at the sentences and their classes. We have kept the training sentences in file example_train.csv. Test sentences have been put in the file example_test.csv.

In [1]:
import numpy as np
import pandas as pd
import sklearn

In [67]:
# training data
train_docs = pd.read_csv('movie_review_train.csv') 
train_docs.head()

,class,text
0,Pos,a common complaint amongst film critics is ...
1,Pos,whew this film oozes energy the kind of b...
2,Pos,steven spielberg s amistad which is bas...
3,Pos,he has spent his entire life in an awful litt...
4,Pos,being that it is a foreign language film with...


In [68]:
#df.column_name.value_counts() - gives no. of unique inputs in that columns
train_docs["class"].value_counts()

Pos    800
Neg    800
Name: class, dtype: int64

In [69]:
pos_negs=train_docs["class"].value_counts()
pos_negs

Pos    800
Neg    800
Name: class, dtype: int64

In [70]:
# convert label to a numerical variable
train_docs['class'] = train_docs['class'].map({'Neg':0, 'Pos':1})
train_docs.head()

,class,text
0,1,a common complaint amongst film critics is ...
1,1,whew this film oozes energy the kind of b...
2,1,steven spielberg s amistad which is bas...
3,1,he has spent his entire life in an awful litt...
4,1,being that it is a foreign language film with...


- Let's now split the dataframe into X and y labels.

In [71]:
X=train_docs.text
y=train_docs['class']
print(X.shape)
print(y.shape)

(1600,)
(1600,)


In [72]:
# convert the df to a numpy array 
train_array = train_docs.values

# split X and y
#X_train = train_array[:,1]
#y_train = train_array[:,0]
#y_train = y_train.astype('int') # sklearn needs y as integers

print("X_train")
print(X_train.shape)
print("y_train")
print(y_train.shape)

X_train
(1600,)
y_train
(1600,)


# Creating the Bag of Words Representation
We now have to convert the data into a format which can be used for training the model. We'll use the bag of words representation for each sentence (document).

Imagine breaking X in individual words and putting them all in a bag. Then we pick all the unique words from the bag one by one and make a dictionary of unique words.

This is called vectorization of words. We have the class CountVectorizer() in scikit learn to vectorize the words.

In [73]:
# create an object of CountVectorizer() class 
from sklearn.feature_extraction.text import CountVectorizer 
# help(CountVectorizer)

In [74]:
vec = CountVectorizer()

In [76]:
# fit the vectorizer on training data 
vec.fit(X_train)
vec.vocabulary_

{'common': 6343,
 'complaint': 6419,
 'amongst': 1379,
 'film': 11923,
 'critics': 7439,
 'is': 16875,
 'why': 35407,
 'aren': 1839,
 'there': 32212,
 'more': 20840,
 'literate': 18743,
 'scripts': 28117,
 'available': 2351,
 'quiz': 25446,
 'show': 28864,
 'gives': 13369,
 'signs': 29019,
 'of': 22167,
 'hope': 15195,
 'that': 32168,
 'the': 32171,
 'art': 1958,
 'writing': 35819,
 'isn': 16903,
 'dead': 7967,
 'in': 15974,
 'hollywood': 15084,
 'and': 1457,
 'we': 35110,
 'need': 21440,
 'not': 21849,
 'only': 22295,
 'look': 18910,
 'to': 32570,
 'independent': 16117,
 'films': 11942,
 'for': 12399,
 'thoughtful': 32307,
 'content': 6831,
 'paul': 23237,
 'attanasio': 2205,
 'script': 28111,
 'takes': 31712,
 'what': 35283,
 'could': 7132,
 'have': 14515,
 'been': 2993,
 'tepid': 32066,
 'thriller': 32338,
 'scandals': 27828,
 'late': 18154,
 '50s': 313,
 'delivers': 8278,
 'telling': 31991,
 'parable': 23006,
 'about': 480,
 'emptiness': 10417,
 'post': 24373,
 'war': 34976,
 'amer

Countvectorizer() has converted the documents into a set of unique words alphabetically sorted and indexed.

**Stop Words**

We can see a few trivial words such as 'and','is','of', etc. These words don't really make any difference in classyfying a document. These are called stop words. So we would like to get rid of them.

We can remove them by passing a parameter stop_words='english' while instantiating Countvectorizer() as follows:

In [77]:
# fitting the vectorizer on training data again
# removing the stop words this time
vec = CountVectorizer(stop_words='english')
vec.fit(X_train)
vec.vocabulary_

{'common': 6284,
 'complaint': 6360,
 'film': 11832,
 'critics': 7378,
 'aren': 1810,
 'literate': 18603,
 'scripts': 27918,
 'available': 2319,
 'quiz': 25250,
 'gives': 13262,
 'signs': 28809,
 'hope': 15074,
 'art': 1928,
 'writing': 35521,
 'isn': 16771,
 'dead': 7904,
 'hollywood': 14963,
 'need': 21281,
 'look': 18770,
 'independent': 15988,
 'films': 11851,
 'thoughtful': 32060,
 'content': 6771,
 'paul': 23045,
 'attanasio': 2173,
 'script': 27912,
 'takes': 31486,
 'tepid': 31839,
 'thriller': 32090,
 'scandals': 27629,
 'late': 18017,
 '50s': 313,
 'delivers': 8215,
 'telling': 31765,
 'parable': 22815,
 'emptiness': 10340,
 'post': 24178,
 'war': 34707,
 'american': 1334,
 'dream': 9554,
 'golden': 13449,
 'bubble': 4285,
 'surrounds': 31156,
 'protects': 24849,
 'tv': 33056,
 'networks': 21378,
 'sponsors': 29948,
 'riddled': 26743,
 'symbols': 31370,
 '58': 328,
 'chrysler': 5614,
 'radio': 25314,
 'announcement': 1534,
 'sputnik': 30053,
 'heavy': 14532,
 'handed': 14185,

Notice that the vocabulary has reduced to 12 from 15. Another way of printing the 'vocabulary' is as follows:

In [78]:
# printing feature names
#print(vec.get_feature_names())
print(len(vec.get_feature_names()))

35858


/usr/local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [79]:
vec1 = CountVectorizer(stop_words='english', min_df=.03, max_df=.8) 
vec1.fit(X_train)
vec1.vocabulary_

{'common': 264,
 'critics': 323,
 'aren': 78,
 'available': 101,
 'gives': 618,
 'hope': 693,
 'art': 81,
 'writing': 1632,
 'isn': 753,
 'dead': 342,
 'hollywood': 690,
 'need': 970,
 'look': 853,
 'films': 549,
 'content': 287,
 'paul': 1037,
 'script': 1248,
 'takes': 1429,
 'thriller': 1471,
 'late': 805,
 'delivers': 357,
 'telling': 1449,
 'post': 1089,
 'war': 1571,
 'american': 59,
 'dream': 413,
 'tv': 1512,
 'radio': 1145,
 'heavy': 673,
 'direction': 386,
 'robert': 1205,
 'performances': 1043,
 'john': 766,
 'rob': 1204,
 'perfectly': 1041,
 'usually': 1537,
 'quality': 1136,
 'sets': 1272,
 'camera': 192,
 'work': 1618,
 'recent': 1165,
 'century': 213,
 'period': 1044,
 'pieces': 1056,
 'years': 1638,
 'old': 1001,
 'images': 716,
 'true': 1502,
 'era': 464,
 'generation': 605,
 'gone': 625,
 '15': 4,
 'world': 1623,
 'themes': 1461,
 'good': 626,
 'life': 831,
 'family': 508,
 'match': 894,
 'father': 521,
 'fame': 506,
 'audience': 99,
 'appear': 72,
 'familiar': 507,
 

So our final dictionary is made of 12 words (after discarding the stop words). Now, to do classification, we need to represent all the documents with these words (or tokens) as features.

Every document will be converted into a feature vector representing presence of these words in that document. Let's convert each of our training documents in to a feature vector.

In [80]:
print(len(vec1.vocabulary_))

1643


In [81]:
# another way of representing the features
X_transformed = vec1.transform(X_train)
X_transformed

<1600x1643 sparse matrix of type '<class 'numpy.int64'>'
	with 217396 stored elements in Compressed Sparse Row format>

You can see X_tranformed is a 5 x 12 sparse matrix. It has 5 rows for each of our 5 documents and 12 columns each for one word of the dictionary which we just created. Let us print X_transformed.

In [82]:
print(X_transformed)

  (0, 4)	1
  (0, 59)	2
  (0, 72)	1
  (0, 78)	1
  (0, 81)	1
  (0, 99)	3
  (0, 101)	1
  (0, 192)	1
  (0, 211)	1
  (0, 213)	1
  (0, 220)	1
  (0, 264)	2
  (0, 287)	1
  (0, 316)	1
  (0, 323)	1
  (0, 328)	1
  (0, 340)	1
  (0, 342)	1
  (0, 357)	1
  (0, 386)	1
  (0, 413)	2
  (0, 445)	2
  (0, 464)	1
  (0, 503)	1
  (0, 506)	1
  :	:
  (1599, 1247)	1
  (1599, 1258)	2
  (1599, 1267)	1
  (1599, 1271)	1
  (1599, 1331)	1
  (1599, 1335)	1
  (1599, 1339)	1
  (1599, 1366)	3
  (1599, 1371)	1
  (1599, 1375)	1
  (1599, 1379)	1
  (1599, 1413)	1
  (1599, 1421)	1
  (1599, 1429)	2
  (1599, 1447)	1
  (1599, 1501)	1
  (1599, 1533)	1
  (1599, 1550)	1
  (1599, 1555)	1
  (1599, 1556)	1
  (1599, 1570)	1
  (1599, 1579)	2
  (1599, 1589)	1
  (1599, 1609)	1
  (1599, 1616)	1


This representation can be understood as follows:

Consider first 4 rows of the output: (0,2), (0,5), (0,7) and (0,11). It says that the first document (index 0) has 7th , 2nd , 5th and 11th 'word' present in the document, and that they appear only once in the document- indicated by the right hand column entry.

Similarly, consider the entry (4,4) (third from bottom). It says that the fifth document has the fifth word present twice. Indeed, the 5th word('good') appears twice in the 5th document.

In real problems, you often work with large documents and vocabularies, and each document contains only a few words in the vocabulary. So it would be a waste of space to store the vocabulary in a typical dataframe, since most entries would be zero. Also, matrix products, additions etc. are much faster with sparse matrices. That's why we use sparse matrices to store the data.

Let us convert this sparse matrix into a more easily interpretable array:

In [83]:
# converting transformed matrix back to an array
# note the high number of zeros
X_transformed.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

To make the dataset more readable, let us examine the vocabulary and the document-term matrix together in a pandas dataframe. The way to convert a matrix into a dataframe is pd.DataFrame(matrix, columns=columns).

In [84]:
# converting matrix to dataframe
pd.DataFrame(X_transformed.toarray(), 
             columns=vec1.get_feature_names())

,000,10,100,13,15,1995,1996,1997,1998,1999,...,written,wrong,wrote,yeah,year,years,yes,york,young,younger
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,1,0,0,0
1596,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
1597,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1598,0,0,0,0,0,2,1,0,0,0,...,1,1,0,0,0,0,0,0,0,0


This table shows how many times a particular word occurs in document. In other words, this is a frequency table of the words.

A corpus of documents can thus be represented by a matrix with one row per document and one column per token (e.g. word) occurring in the corpus.

Let's now import and transform the test data as well.

In [85]:
# test data
test_docs = pd.read_csv('movie_review_test.csv') 
test_docs

,class,text
0,Pos,films adapted from comic books have had plent...
1,Pos,every now and then a movie comes along from a...
2,Pos,you ve got mail works alot better than it des...
3,Pos,jaws is a rare film that grabs your atte...
4,Pos,moviemaking is a lot like being the general m...
...,...,...
395,Neg,one of the first films of 1999 is this mtv pi...
396,Neg,susan granger s review of america s sweethe...
397,Neg,susan granger s review of jeepers creepers ...
398,Neg,this independent film written and directed by...


In [86]:
# convert label to a numerical variable
test_docs['class'] = test_docs['class'].map({'Neg':0, 'Pos':1})
test_docs

,class,text
0,1,films adapted from comic books have had plent...
1,1,every now and then a movie comes along from a...
2,1,you ve got mail works alot better than it des...
3,1,jaws is a rare film that grabs your atte...
4,1,moviemaking is a lot like being the general m...
...,...,...
395,0,one of the first films of 1999 is this mtv pi...
396,0,susan granger s review of america s sweethe...
397,0,susan granger s review of jeepers creepers ...
398,0,this independent film written and directed by...


In [109]:
# convert to numpy array
test_numpy_array = test_docs.values

# split into X and y
X_test = test_numpy_array[:,1]
y_test = test_numpy_array[:,0]
y_test = y_test.astype('int') # sklearn needs y as integers

print("X_test")
print(X_test.shape)
print("y_test")
print(y_test.shape)

X_test
(400,)
y_test
(400,)


In [110]:
# transform the test data
# note that you *never* fit on test data, only on training data
# and only transform the test data
X_test_transformed = vec1.transform(X_test)
X_test_transformed

<400x1643 sparse matrix of type '<class 'numpy.int64'>'
	with 51663 stored elements in Compressed Sparse Row format>

In [111]:
# convert to non-sparse array
X_test=X_test_transformed.toarray()
X_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 2, 0]])

Let us summarise all we have done till now:

vect.fit(train) learns the vocabulary of the training data
vect.transform(train) uses the fitted vocabulary to build a document-term matrix from the training data
vect.transform(test) uses the fitted vocabulary to build a document-term matrix from the testing data (and ignores tokens it hasn't seen before)
# 2. Building the Model: Multinomial Naive Bayes

# 3. Building the Model: Bernoulli Naive Bayes

In [112]:
from sklearn.naive_bayes import BernoulliNB

# instantiating bernoulli NB class
bnb=BernoulliNB()

# fitting the model
bnb.fit(X_transformed, y_train)

# also works
# bnb.fit(X_transformed.toarray(), y_train)

# predict class
y_pred_class = bnb.predict(X_test)

# predict probability
y_pred_proba =bnb.predict_proba(X_test_transformed)


In [113]:
# printing the overall accuracy
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.79

In the next notebook, we will use Multinomial and Bernoulli Naive Bayes to solve an interesting real problem - classifying SMSes as spam or ham. We'll also see how to decide the optimal cutoff probability and evaluate the model.

In [114]:
# confusion matrix
metrics.confusion_matrix(y_test, y_pred_class)
# help(metrics.confusion_matrix)

array([[177,  23],
       [ 61, 139]])